In [1]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import astropy.units as u
import numpy as np

In [2]:
mh = '+0.0'
CtoO = '2.5'

In [3]:
filename_db="/Users/sagnickmukherjee/Documents/software/picaso-dev/reference/opacities/ck_cx_cont_opacities.db"
ck_db='/Users/sagnickmukherjee/Documents/software/picaso-dev/reference/opacities/ck_db/m'+mh+'_co'+CtoO+'.data.196'
opacity_ck = jdi.opannection(ck=True, ck_db=ck_db,filename_db=filename_db)

In [4]:
w_star,f_star = np.loadtxt("temp_dir_test/sol_spec00f.dat",usecols=[0,1],unpack=True) # W/m^2/micron at 1AU
#convert to erg/s/cm^2/A
f_star=f_star*1e3/1e4 # this is now erg/s/cm^2/A # this is flux at 1 AU
r_sun =0.00465047 # AU
# so surface brightness is

f_star = f_star * (1.0/r_sun)**2 

np.savetxt("temp_dir_test/sol_flam.txt",np.transpose([w_star,f_star]))


In [5]:
semi_major = 0.05 # AU
T_eff = 300
T_star = 6000
grav = 25

star_file = "temp_dir_test/sol_flam.txt"

cl_run = jdi.inputs(calculation="planet", climate = True)


cl_run.star(opacity_ck, temp =T_star,metal =0.25, logg =4.0, radius = 1, radius_unit=u.R_sun,semi_major= semi_major , semi_major_unit = u.AU)#opacity db, pysynphot database, temp, metallicity, logg
#cl_run.star(opacity_ck, radius = 1, radius_unit=u.R_sun,semi_major= semi_major , semi_major_unit = u.AU,filename= star_file, w_unit='um', f_unit='FLAM')
## output stellar radiation from this is ready to be used in RT in our climate code just after distance and radius adjustment

cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)'))
cl_run.T_eff(Teff=T_eff)
cl_run.setup_climate_data()



In [6]:
nlevel = 91
pressure=np.logspace(np.log10(0.000001000 ),np.log10(1000.0 ),nlevel)
temp_guess = np.zeros_like(pressure) + 300.0

nofczns = 1
nstr = np.array([0,80,89,0,0,0])
rfacv = 0.5
rfaci =1.0
cloudy = False


cl_run.inputs_climate(temp_guess= temp_guess, pressure= pressure, nstr = nstr, nofczns = nofczns , rfacv = rfacv, rfaci = rfaci, cloudy = cloudy, mh = mh, CtoO = None)

In [7]:
pressure , temp, dtdp, nstr_new, flux_plus_final = cl_run.run_climate_model(opacity_ck)

Iteration number  0 , min , max temp  300.03515067577257 687.3158604745947 , flux balance  -192.86196300128563
Iteration number  1 , min , max temp  302.0188507889139 2301.662317918749 , flux balance  -192.6845027527929
Iteration number  2 , min , max temp  348.8150466193314 2407.3967770400022 , flux balance  -185.2443514912681
Iteration number  3 , min , max temp  608.4232818048274 3083.1161268597602 , flux balance  -129.4550504989057
Iteration number  4 , min , max temp  662.3719452931387 3164.0418182216135 , flux balance  -98.4065159620844
Iteration number  5 , min , max temp  820.8504129825933 3402.5550688376834 , flux balance  59.096136995764816
Iteration number  6 , min , max temp  784.7234265323374 3354.1452739172832 , flux balance  6.122467580142192
Iteration number  7 , min , max temp  782.0603497321941 3349.9028288683908 , flux balance  0.11898209805569583
Iteration number  8 , min , max temp  782.0463302159554 3349.792538970927 , flux balance  0.00016765607095378708
Converge

KeyboardInterrupt: 

In [ ]:
filename= "/Users/sagnickmukherjee/Documents/software/picaso-dev/RESULTS_benchmarking/tp_file/tp_"+str(T_eff)+"_grav_"+str(grav)+"_mh_"+mh+"_sm_"+str(semi_major)+".dat"
np.savetxt(filename, np.transpose([pressure,temp]), header= str("nstr "+str(nstr_new)+" T_eff "+str(T_eff)+" grav "+ str(grav)+" T_star "+str(T_star)+" semi-major[AU] "+str(semi_major)))                                                                 